In [1]:
from dotenv import load_dotenv
load_dotenv()  # this loads .env into environment variables
import os

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## for Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
model = ChatGroq(model="gemma2-9b-it", temperature=0, groq_api_key=os.environ["GROQ_API_KEY"])
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001B5AAF3F0D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001B5AAF3CC70>, model_name='gemma2-9b-it', temperature=1e-08, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

msg = [
    SystemMessage(content="Translate the following from English to french"),
    HumanMessage(content="Hello, How are you?")
]

llm_result = model.invoke(msg)
llm_result

AIMessage(content='Bonjour, comment allez-vous ? \n\n\nYou can also use the more informal:\n\n**Salut, comment vas-tu ?** \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 22, 'total_tokens': 53, 'completion_time': 0.056363636, 'prompt_time': 0.002129689, 'queue_time': 0.249425291, 'total_time': 0.058493325}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--c7cf3c2a-b6a2-4dcb-a6b5-bdb7c2aeb56d-0', usage_metadata={'input_tokens': 22, 'output_tokens': 31, 'total_tokens': 53})

In [4]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser.invoke(llm_result)

'Bonjour, comment allez-vous ? \n\n\nYou can also use the more informal:\n\n**Salut, comment vas-tu ?** \n'

In [5]:
#### Using LCEL - chain the components
chain = model | parser
chain.invoke(msg)

'Bonjour, comment allez-vous ? \n\n\nYou can also use the more informal:\n\n**Salut, comment vas-tu ?** \n'

In [6]:
### Prompt templates 
from langchain_core.prompts import ChatPromptTemplate
generic_template = "Translate the following into {language}:"
prompt = ChatPromptTemplate.from_messages(
    [("system", generic_template),
     ("user", "{input_text}")
     ]
)

prompt

ChatPromptTemplate(input_variables=['input_text', 'language'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='Translate the following into {language}:'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input_text'], input_types={}, partial_variables={}, template='{input_text}'), additional_kwargs={})])

In [7]:
prompt_input_template = prompt.invoke({"language": "French", "input_text": "Hello"})
prompt_input_template

ChatPromptValue(messages=[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})])

In [8]:
prompt_input_template.to_messages()

[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [9]:
chain = prompt | model | parser
chain.invoke({"language": "French", "input_text": "Hello"})

'Bonjour \n'